In [3]:
import streamlit as st
import requests
import pandas as pd
import plotly.express as px
import datetime
from datetime import datetime




In [4]:
API_KEY=st.secrets['ESIOS_API_KEY']

In [5]:
def download_esios_id(id,fecha_ini,fecha_fin,agrupacion):
                       token = API_KEY
                       
                       cab = dict()
                       cab ['x-api-key']= token
                       url_id = 'https://api.esios.ree.es/indicators'
                       url=f'{url_id}/{id}?geo_ids[]=8741&time_agg=average&start_date={fecha_ini}T00:00:00&end_date={fecha_fin}T23:59:59&time_trunc={agrupacion}'
                       print(url)
                       response = requests.get(url, headers=cab).json()
                       print(response)

                       return response
                       

In [6]:
id='10391'
fecha_ini='2024-01-01'
fecha_fin='2024-12-31'
agrupacion='day'


In [7]:
datos_origen =download_esios_id(id,fecha_ini,fecha_fin,agrupacion)


https://api.esios.ree.es/indicators/10391?geo_ids[]=8741&time_agg=average&start_date=2024-01-01T00:00:00&end_date=2024-12-31T23:59:59&time_trunc=day
{'indicator': {'name': 'Término de facturación de energía activa del PVPC 2.0TD suma componentes', 'short_name': '2.0TD suma componentes', 'id': 10391, 'composited': True, 'step_type': 'linear', 'disaggregated': True, 'magnitud': [{'name': 'Precio', 'id': 23}], 'tiempo': [{'name': 'Hora', 'id': 4}], 'geos': [{'geo_id': 8741, 'geo_name': 'Península'}], 'values_updated_at': '2024-01-09T16:30:02.000+01:00', 'values': [{'value': 90.572083, 'datetime': '2024-01-01T00:00:00.000+01:00', 'datetime_utc': '2023-12-31T23:00:00Z', 'tz_time': '2023-12-31T23:00:00.000Z', 'geo_id': 8741, 'geo_name': 'Península'}, {'value': 120.48, 'datetime': '2024-01-02T00:00:00.000+01:00', 'datetime_utc': '2024-01-01T23:00:00Z', 'tz_time': '2024-01-01T23:00:00.000Z', 'geo_id': 8741, 'geo_name': 'Península'}, {'value': 130.6025, 'datetime': '2024-01-03T00:00:00.000+01:0

In [8]:
datos=pd.DataFrame(datos_origen['indicator']['values'])
datos

,value,datetime,datetime_utc,tz_time,geo_id,geo_name
0,90.572083,2024-01-01T00:00:00.000+01:00,2023-12-31T23:00:00Z,2023-12-31T23:00:00.000Z,8741,Península
1,120.480000,2024-01-02T00:00:00.000+01:00,2024-01-01T23:00:00Z,2024-01-01T23:00:00.000Z,8741,Península
2,130.602500,2024-01-03T00:00:00.000+01:00,2024-01-02T23:00:00Z,2024-01-02T23:00:00.000Z,8741,Península
3,165.828750,2024-01-04T00:00:00.000+01:00,2024-01-03T23:00:00Z,2024-01-03T23:00:00.000Z,8741,Península
4,143.957500,2024-01-05T00:00:00.000+01:00,2024-01-04T23:00:00Z,2024-01-04T23:00:00.000Z,8741,Península
...,...,...,...,...,...,...
188,83.943333,2024-07-07T00:00:00.000+02:00,2024-07-06T22:00:00Z,2024-07-06T22:00:00.000Z,8741,Península
189,144.599583,2024-07-08T00:00:00.000+02:00,2024-07-07T22:00:00Z,2024-07-07T22:00:00.000Z,8741,Península
190,138.257500,2024-07-09T00:00:00.000+02:00,2024-07-08T22:00:00Z,2024-07-08T22:00:00.000Z,8741,Península
191,150.919583,2024-07-10T00:00:00.000+02:00,2024-07-09T22:00:00Z,2024-07-09T22:00:00.000Z,8741,Península


In [9]:
#datos=datos[datos['geo_name'].str.contains('Península')]

In [10]:
datos = (datos
         .assign(datetime=lambda vh_: pd #formateamos campo fecha, desde un str con diferencia horaria a un naive
                      .to_datetime(vh_['datetime'],utc=True)  # con la fecha local
                      .dt
                      .tz_convert('Europe/Madrid')
                      .dt
                      .tz_localize(None)
                ) 
             #.drop(['datetime','datetime_utc','tz_time','geo_id','geo_name'],
             #      axis=1) #eliminamos campos
             
             .loc[:,['datetime','value']]
             )

datos

,datetime,value
0,2024-01-01,90.572083
1,2024-01-02,120.480000
2,2024-01-03,130.602500
3,2024-01-04,165.828750
4,2024-01-05,143.957500
...,...,...
188,2024-07-07,83.943333
189,2024-07-08,144.599583
190,2024-07-09,138.257500
191,2024-07-10,150.919583


In [11]:
meses = {
    'January': 'Enero',
    'February': 'Febrero',
    'March': 'Marzo',
    'April': 'Abril',
    'May': 'Mayo',
    'June': 'Junio',
    'July': 'Julio',
    'August': 'Agosto',
    'September': 'Septiembre',
    'October': 'Octubre',
    'November': 'Noviembre',
    'December': 'Diciembre'
}

### Esta es la tabla de valores horarios tratada

In [12]:
#datos['fecha']=datos['datetime'].dt.strftime('%d/%m/%Y')
#datos['fecha']=pd.to_datetime(datos['fecha'],format='%d/%m/%Y')
datos['fecha']=datos['datetime'].dt.date
#datos['fecha']=pd.to_datetime(datos['fecha']).dt.date
datos['hora']=datos['datetime'].dt.hour
datos['dia']=datos['datetime'].dt.day
datos['mes']=datos['datetime'].dt.month
datos['año']=datos['datetime'].dt.year
datos.set_index('datetime', inplace=True)
datos

,value,fecha,hora,dia,mes,año
datetime,,,,,,
2024-01-01,90.572083,2024-01-01,0,1,1,2024
2024-01-02,120.480000,2024-01-02,0,2,1,2024
2024-01-03,130.602500,2024-01-03,0,3,1,2024
2024-01-04,165.828750,2024-01-04,0,4,1,2024
2024-01-05,143.957500,2024-01-05,0,5,1,2024
...,...,...,...,...,...,...
2024-07-07,83.943333,2024-07-07,0,7,7,2024
2024-07-08,144.599583,2024-07-08,0,8,7,2024
2024-07-09,138.257500,2024-07-09,0,9,7,2024


In [13]:
datos.dtypes

value    float64
fecha     object
hora       int32
dia        int32
mes        int32
año        int32
dtype: object

In [14]:
def obtener_fecha_reg_max():
    
    ultimo_registro= datos['fecha'].max()
    
    return ultimo_registro

In [56]:
ultimo_registro=obtener_fecha_reg_max()
ultimo_registro

datetime.date(2024, 7, 11)

In [16]:
primer_registro=datos['fecha'].min()

In [17]:
def obtener_dias():
    dias=(ultimo_registro-primer_registro).days
    
    return dias

In [55]:
fecha_hoy=datetime.today().date()
fecha_hoy

datetime.date(2024, 7, 11)

In [19]:
hora=datetime.now().hour
hora

8


if hora>14:
    st.rerun()
    flag=False

In [20]:
valor_minimo_horario=datos['value'].min()
valor_maximo_diario=datos['value'].max()
valor_minimo_horario,valor_maximo_diario

(44.445417, 183.964167)

### Copia para escala de colores

datos_horarios=datos
datos_horarios


### Agrupación por días

In [21]:
datos_dias=datos.reset_index()
datos_dias

,datetime,value,fecha,hora,dia,mes,año
0,2024-01-01,90.572083,2024-01-01,0,1,1,2024
1,2024-01-02,120.480000,2024-01-02,0,2,1,2024
2,2024-01-03,130.602500,2024-01-03,0,3,1,2024
3,2024-01-04,165.828750,2024-01-04,0,4,1,2024
4,2024-01-05,143.957500,2024-01-05,0,5,1,2024
...,...,...,...,...,...,...,...
188,2024-07-07,83.943333,2024-07-07,0,7,7,2024
189,2024-07-08,144.599583,2024-07-08,0,8,7,2024
190,2024-07-09,138.257500,2024-07-09,0,9,7,2024
191,2024-07-10,150.919583,2024-07-10,0,10,7,2024


In [22]:
datos_dia=datos.drop(columns=['hora'])
datos_dia['fecha']=pd.to_datetime(datos['fecha'],format='%d/%m/%Y')
#datos_dia['fecha']=datos_dia['fecha'].dt.strftime('%d/%m/%Y')


datos_dia=datos_dia.resample('D').mean()
datos_dia['value']=datos_dia['value'].round(2)

datos_dia[['dia','mes','año']]=datos_dia[['dia','mes','año']].astype(int)


In [23]:
datos_dia

,value,fecha,dia,mes,año
datetime,,,,,
2024-01-01,90.57,2024-01-01,1,1,2024
2024-01-02,120.48,2024-01-02,2,1,2024
2024-01-03,130.60,2024-01-03,3,1,2024
2024-01-04,165.83,2024-01-04,4,1,2024
2024-01-05,143.96,2024-01-05,5,1,2024
...,...,...,...,...,...
2024-07-07,83.94,2024-07-07,7,7,2024
2024-07-08,144.60,2024-07-08,8,7,2024
2024-07-09,138.26,2024-07-09,9,7,2024


In [24]:
valor_minimo_diario=datos_dia['value'].min()
valor_maximo_diario=datos_dia['value'].max()
valor_minimo_diario,valor_maximo_diario

(44.45, 183.96)

### Agrupación por meses

In [25]:
datos

,value,fecha,hora,dia,mes,año
datetime,,,,,,
2024-01-01,90.572083,2024-01-01,0,1,1,2024
2024-01-02,120.480000,2024-01-02,0,2,1,2024
2024-01-03,130.602500,2024-01-03,0,3,1,2024
2024-01-04,165.828750,2024-01-04,0,4,1,2024
2024-01-05,143.957500,2024-01-05,0,5,1,2024
...,...,...,...,...,...,...
2024-07-07,83.943333,2024-07-07,0,7,7,2024
2024-07-08,144.599583,2024-07-08,0,8,7,2024
2024-07-09,138.257500,2024-07-09,0,9,7,2024


In [26]:
datos_mes=datos.drop(columns=['fecha','hora', 'dia'])
datos_mes=datos_mes.resample('M').mean()
datos_mes['value']=datos_mes['value'].round(2)

datos_mes[['mes','año']]=datos_mes[['mes','año']].astype(int)
datos_mes

,value,mes,año
datetime,,,
2024-01-31,139.50,1,2024
2024-02-29,106.64,2,2024
2024-03-31,89.92,3,2024
2024-04-30,85.58,4,2024
2024-05-31,99.67,5,2024
2024-06-30,117.02,6,2024
2024-07-31,127.01,7,2024


### Definimos la escala y sus límites

In [27]:
datos_limites = {
    'rango': [-10,20.01,40.01,60.01,80.01,100.01,10000],
    'valor_asignado': ['muy bajo', 'bajo','medio','alto','muy alto','chungo','xtrem'],
}

In [28]:
df_limites=pd.DataFrame(datos_limites)
df_limites

,rango,valor_asignado
0,-10.00,muy bajo
1,20.01,bajo
2,40.01,medio
3,60.01,alto
4,80.01,muy alto
5,100.01,chungo
6,10000.00,xtrem


In [29]:
#etiquetas = df_limites['valor_asignado'][:-1]
etiquetas = df_limites['valor_asignado'][:-1]
etiquetas

0    muy bajo
1        bajo
2       medio
3        alto
4    muy alto
5      chungo
Name: valor_asignado, dtype: object

### Añadimos la columna escala a todas las tablas

datos_horarios['escala']=pd.cut(datos_horarios['value'],bins=df_limites['rango'],labels=etiquetas,right=True)
#datos_horarios['escala']=pd.cut(datos_horarios['value'],bins=df_limites['rango'],labels=etiquetas)
#datos_horarios['escala']=pd.cut(datos_horarios['value'],bins=df_limites['rango'],labels=df_limites['valor_asignado'],right=False)
datos_horarios

lista_escala=datos_horarios['escala'].unique()
lista_escala

In [30]:
datos_dia['escala']=pd.cut(datos_dia['value'],bins=df_limites['rango'],labels=etiquetas,right=False)
datos_dia

,value,fecha,dia,mes,año,escala
datetime,,,,,,
2024-01-01,90.57,2024-01-01,1,1,2024,muy alto
2024-01-02,120.48,2024-01-02,2,1,2024,chungo
2024-01-03,130.60,2024-01-03,3,1,2024,chungo
2024-01-04,165.83,2024-01-04,4,1,2024,chungo
2024-01-05,143.96,2024-01-05,5,1,2024,chungo
...,...,...,...,...,...,...
2024-07-07,83.94,2024-07-07,7,7,2024,muy alto
2024-07-08,144.60,2024-07-08,8,7,2024,chungo
2024-07-09,138.26,2024-07-09,9,7,2024,chungo


In [31]:
datos_mes['escala']=pd.cut(datos_mes['value'],bins=df_limites['rango'],labels=etiquetas,right=False)
datos_mes

,value,mes,año,escala
datetime,,,,
2024-01-31,139.50,1,2024,chungo
2024-02-29,106.64,2,2024,chungo
2024-03-31,89.92,3,2024,muy alto
2024-04-30,85.58,4,2024,muy alto
2024-05-31,99.67,5,2024,muy alto
2024-06-30,117.02,6,2024,chungo
2024-07-31,127.01,7,2024,chungo


### Definimos los colores según la escala

In [32]:
colores = {
    'muy bajo': 'lightgreen',
    'bajo': 'green',
    'medio': 'blue',
    'alto': 'orange',
    'muy alto': 'red',
    'chungo': 'purple',
    'xtrem':'black'
}

datos_horarios['color']=datos_horarios['escala'].map(colores)
datos_horarios

In [33]:
datos_dia['color']=datos_dia['escala'].map(colores)
datos_dia

,value,fecha,dia,mes,año,escala,color
datetime,,,,,,,
2024-01-01,90.57,2024-01-01,1,1,2024,muy alto,red
2024-01-02,120.48,2024-01-02,2,1,2024,chungo,purple
2024-01-03,130.60,2024-01-03,3,1,2024,chungo,purple
2024-01-04,165.83,2024-01-04,4,1,2024,chungo,purple
2024-01-05,143.96,2024-01-05,5,1,2024,chungo,purple
...,...,...,...,...,...,...,...
2024-07-07,83.94,2024-07-07,7,7,2024,muy alto,red
2024-07-08,144.60,2024-07-08,8,7,2024,chungo,purple
2024-07-09,138.26,2024-07-09,9,7,2024,chungo,purple


### Obtenemos la escala ordenada al revés para el gráfico horario

In [34]:
valor_asignado_a_rango = {row['valor_asignado']: row['rango'] for _, row in df_limites.iterrows()}

escala_horaria=['alto', 'medio', 'bajo', 'muy bajo', 'muy alto', 'chungo']
escala_horaria

escala_ordenada_hora = sorted(escala_horaria, key=lambda x: valor_asignado_a_rango[x], reverse=True)
escala_ordenada_hora

datos_horarios['color']=datos_horarios['escala'].map(colores)
datos_horarios

datos_horarios['escala']=pd.Categorical(datos_horarios['escala'],categories=escala_ordenada_hora, ordered=True)
datos_horarios

### Obtenemos la escala ordenada al reves para el gráfico diario

In [35]:
escala_dia=datos_dia['escala'].unique()
escala_dia

['muy alto', 'chungo', 'medio', 'alto']
Categories (6, object): ['muy bajo' < 'bajo' < 'medio' < 'alto' < 'muy alto' < 'chungo']

In [36]:
escala_ordenada_dia = sorted(escala_dia, key=lambda x: valor_asignado_a_rango[x], reverse=True)
escala_ordenada_dia

['chungo', 'muy alto', 'alto', 'medio']

In [37]:
datos_mes['color']=datos_mes['escala'].map(colores)
datos_mes

,value,mes,año,escala,color
datetime,,,,,
2024-01-31,139.50,1,2024,chungo,purple
2024-02-29,106.64,2,2024,chungo,purple
2024-03-31,89.92,3,2024,muy alto,red
2024-04-30,85.58,4,2024,muy alto,red
2024-05-31,99.67,5,2024,muy alto,red
2024-06-30,117.02,6,2024,chungo,purple
2024-07-31,127.01,7,2024,chungo,purple


### Obtenemos la escala ordenada al reves para el gráfico mensual

In [38]:
escala_mes= datos_mes['escala'].unique()
escala_mes

['chungo', 'muy alto']
Categories (6, object): ['muy bajo' < 'bajo' < 'medio' < 'alto' < 'muy alto' < 'chungo']

In [39]:
escala_ordenada_mes = sorted(escala_mes, key=lambda x: valor_asignado_a_rango[x], reverse=True)
escala_ordenada_mes

['chungo', 'muy alto']

### Esta tabla se usa para el gráfico grande de barras diario

In [40]:

datos_dia['escala']=pd.Categorical(datos_dia['escala'],categories=escala_ordenada_dia, ordered=True)
datos_dia

,value,fecha,dia,mes,año,escala,color
datetime,,,,,,,
2024-01-01,90.57,2024-01-01,1,1,2024,muy alto,red
2024-01-02,120.48,2024-01-02,2,1,2024,chungo,purple
2024-01-03,130.60,2024-01-03,3,1,2024,chungo,purple
2024-01-04,165.83,2024-01-04,4,1,2024,chungo,purple
2024-01-05,143.96,2024-01-05,5,1,2024,chungo,purple
...,...,...,...,...,...,...,...
2024-07-07,83.94,2024-07-07,7,7,2024,muy alto,red
2024-07-08,144.60,2024-07-08,8,7,2024,chungo,purple
2024-07-09,138.26,2024-07-09,9,7,2024,chungo,purple


### Estos datos se usan para el grafico de barras mensual

In [41]:
datos_mes['escala']=pd.Categorical(datos_mes['escala'],categories=escala_ordenada_mes, ordered=True)
datos_mes

,value,mes,año,escala,color
datetime,,,,,
2024-01-31,139.50,1,2024,chungo,purple
2024-02-29,106.64,2,2024,chungo,purple
2024-03-31,89.92,3,2024,muy alto,red
2024-04-30,85.58,4,2024,muy alto,red
2024-05-31,99.67,5,2024,muy alto,red
2024-06-30,117.02,6,2024,chungo,purple
2024-07-31,127.01,7,2024,chungo,purple


### Estos datos se usan para el quesito

In [42]:
datos_dia_queso=datos_dia.groupby(['escala'])['escala'].count()
datos_dia_queso=datos_dia_queso.reset_index(name='num_dias')
datos_dia_queso


,escala,num_dias
0,chungo,109
1,muy alto,50
2,alto,18
3,medio,16


### Gráfico de barras principal

In [43]:
#datos_dia.reset_index()
#datos_dia.rename(columns={'datetime','fecha'},inplace=True)
datos_dia
#datos_dia['datetime'].rename['fecha']

,value,fecha,dia,mes,año,escala,color
datetime,,,,,,,
2024-01-01,90.57,2024-01-01,1,1,2024,muy alto,red
2024-01-02,120.48,2024-01-02,2,1,2024,chungo,purple
2024-01-03,130.60,2024-01-03,3,1,2024,chungo,purple
2024-01-04,165.83,2024-01-04,4,1,2024,chungo,purple
2024-01-05,143.96,2024-01-05,5,1,2024,chungo,purple
...,...,...,...,...,...,...,...
2024-07-07,83.94,2024-07-07,7,7,2024,muy alto,red
2024-07-08,144.60,2024-07-08,8,7,2024,chungo,purple
2024-07-09,138.26,2024-07-09,9,7,2024,chungo,purple


In [44]:
datos_dia.dtypes

value            float64
fecha     datetime64[ns]
dia                int32
mes                int32
año                int32
escala          category
color           category
dtype: object

In [45]:
def graf_ecv_anual():
    graf_ecv_anual=px.bar(datos_dia, x='fecha', y='value', 
        color='escala',
        color_discrete_map=colores,
        category_orders={'escala':escala_ordenada_dia},
        labels={'value':'precio medio diario €/MWh', 'escala':'escala_cv'},
        title="Precios medios del mercado diario OMIE. Año 2024")
    graf_ecv_anual.update_xaxes(
        showgrid=True
    )
    graf_ecv_anual.update_traces(
        marker_line_width=0
    )

    return graf_ecv_anual

In [46]:
graf_ecv_anual()

In [47]:
def graf_ecv_mensual():
    graf_ecv_mensual=px.bar(datos_mes, x='mes', y='value',
        color='escala',
        color_discrete_map=colores,
        category_orders={'escala':escala_ordenada_mes},
        labels={'value':'precio medio mensual €/MWh', 'escala':'escala_cv'},
        title="Precios medios mensuales. Año 2024"
        )
    #graf_ecv_mensual.update_xaxes(
    #    showgrid=True
    
    #graf_ecv_mensual.update_traces(
    #   marker_line_width=0
    #)

    return graf_ecv_mensual

In [48]:
graf_ecv_mensual()

### Gráfico de queso

In [49]:
def graf_ecv_anual_queso():
    graf_ecv_anual_queso=px.pie(datos_dia_queso, values='num_dias', names='escala',
        color='escala',
        color_discrete_map=colores,
        #marker=dict(colors=colores),
        category_orders={'escala':escala_ordenada_dia},
        labels={'num_dias':'num_dias', 'escala':'escala_cv'},
        title="% y número de días según la Escala CV. Año 2024",
        width=500
        )
    
    return graf_ecv_anual_queso

In [50]:
graf_ecv_anual_queso()

## Cálculo del precio medio del PVPC

In [51]:
def filtrar_mes(mes):
    if mes !=None:
        filtro_mes=datos['mes']==mes
        datos_filtrados_mes=datos[filtro_mes]
    else:
        datos_filtrados_mes=datos
    
    pvpc_media=datos_filtrados_mes['value'].mean()
    
    return datos_filtrados_mes


In [52]:
def calcular_media_pvpc(mes):
    if mes !=None:
        datos_filtrados_mes=datos
    else:
        datos_filtrados_mes=filtrar_mes(mes)
    
    pvpc_media=datos_filtrados_mes['value'].mean()
    #pvpc_media

    return pvpc_media

In [53]:
calcular_media_pvpc(1)

107.61532784455959

In [54]:

datos_filtrados_mes=filtrar_mes(1)
datos_filtrados_mes


,value,fecha,hora,dia,mes,año
datetime,,,,,,
2024-01-01,90.572083,2024-01-01,0,1,1,2024
2024-01-02,120.480000,2024-01-02,0,2,1,2024
2024-01-03,130.602500,2024-01-03,0,3,1,2024
2024-01-04,165.828750,2024-01-04,0,4,1,2024
2024-01-05,143.957500,2024-01-05,0,5,1,2024
2024-01-06,88.500833,2024-01-06,0,6,1,2024
2024-01-07,120.919167,2024-01-07,0,7,1,2024
2024-01-08,178.196667,2024-01-08,0,8,1,2024
2024-01-09,183.964167,2024-01-09,0,9,1,2024


datos_filtrados_dia=datos_filtrados.resample('D').mean()
datos_filtrados_dia

def graf_dia():
    graf_dia=px.line(datos_filtrados_dia,x='fecha',y='value')
    
    return graf_dia

graf_dia()